ils en ont 92 features pour entraîner groupe 52

## Setup

In [ ]:

# Import libraries for data manipulation
import pandas as pd
from summarytools import dfSummary

import numpy as np

from icecream import ic
#ic(my_var) : fait un joli print direct

from ydata_profiling import ProfileReport

# import latexify

# Import libraries for data visualization
import matplotlib.pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats("svg")

plt.style.use('seaborn-v0_8-whitegrid')

import seaborn as sns

import copy as cp

# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer

from statsmodels.graphics.gofplots import ProbPlot

# Import libraries for building linear regression model
from statsmodels.formula.api import ols

# Import the required function
from scipy.stats import pearsonr

from sklearn.preprocessing import OrdinalEncoder


import statsmodels.api as sm

from sklearn.linear_model import LinearRegression

# Import library for preparing data
from sklearn.model_selection import train_test_split

# Import library for data preprocessing
from sklearn.preprocessing import MinMaxScaler

from statsmodels.stats.outliers_influence import variance_inflation_factor

from statsmodels.stats.diagnostic import het_white
from statsmodels.compat import lzip
import statsmodels.stats.api as sms

# Plot q-q plot of residuals
import pylab
import scipy.stats as stats

## R2
from sklearn.metrics import r2_score


from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings("ignore")


####################
# Fonctions : 



def corr_plot(df, upper_tri=None, threshold=None):
    
    plt.figure(figsize = (12, 8))
    cmap = sns.diverging_palette(230, 20, as_cmap = True)

    corr = df.corr()
    mask = None

    if threshold != None:
        corr = corr.mask(np.abs(corr) < threshold, np.nan)

    if upper_tri:
        mask = np.tril(np.ones_like(corr, dtype=bool)) # affichera que le triangle supérieur

    sns.heatmap(corr, mask=mask, annot = True, fmt = '.2f', cmap = cmap)
    plt.show()

    return corr

# RMSE
# @latexify.function
def rmse(predictions, targets):
    return np.sqrt(((targets - predictions) ** 2).mean())

# MAPE
# @latexify.function
def mape(predictions, targets):
    return np.mean(np.abs((targets - predictions)) / targets) * 100

# MAE
# @latexify.function
def mae(predictions, targets):
    return np.mean(np.abs((targets - predictions)))

# Function to check VIF (TOUTES LES FEATURES DOIVENT ETRE EN FLOAT !!)
def checking_vif(train):
    vif = pd.DataFrame()
    vif["feature"] = train.columns

    # Calculating VIF for each feature
    vif["VIF"] = [
        variance_inflation_factor(train.values, i) for i in range(len(train.columns))
    ]
    return vif


In [ ]:
df = pd.read_csv("./data/X_train_Hi5.csv")
df = df.sample(n=10_000, random_state=42)
df.head()

In [ ]:
target = "piezo_groundwater_level_category"

## Preprocessing

In [ ]:
# On tolère 19% de nan dans les colonnes
for proportion_nan_prct in [19]:
    new_df = cp.deepcopy(df)
    for col in new_df.columns:
        if new_df[col].isnull().sum() * 100 / len(df) > proportion_nan_prct:
            new_df.drop(col, axis=1, inplace=True)

# new_df.info()

# Gérer les duplicatas
new_df.drop_duplicates(inplace=True)

# Drop les features indépendantes qui sont ultra corrélées entre elles (id...)
features_to_drop = ["piezo_station_commune_code_insee",
                    "piezo_station_pe_label",
                    "piezo_station_bdlisa_codes",
                    "piezo_station_bss_code",
                    "piezo_station_commune_name",
                    "piezo_station_bss_id",
                    "piezo_bss_code",
                    "piezo_station_update_date",
                    "piezo_qualification",
                    "piezo_continuity_code",
                    "piezo_continuity_name",
                    "piezo_producer_name",
                    "piezo_measure_nature_name",
                    "meteo_name",
                    "hydro_station_code",
                    "hydro_method_code",
                    "hydro_method_label",
                    "insee_med_living_level",
                    "meteo_id",
                    "hydro_qualification_label",
                    "hydro_status_code",
                    "piezo_station_department_name"]

features = new_df.drop(features_to_drop, axis=1)
# features.info()

num_col_features =      ["piezo_station_investigation_depth", 
                        "piezo_station_altitude", 
                        "piezo_station_longitude", 
                        "piezo_station_latitude", 
                        "piezo_producer_code", 
                        "meteo_latitude", 
                        "meteo_longitude", 
                        "meteo_altitude", 
                        "meteo_rain_height", 
                        "meteo_temperature_min", 
                        "meteo_time_tn", 
                        "meteo_temperature_max", 
                        "meteo_time_tx", 
                        "meteo_temperature_avg", 
                        "meteo_temperature_avg_threshold",
                        "meteo_frost_duration", 
                        "meteo_amplitude_tn_tx", 
                        "meteo_temperature_avg_tntm", 
                        "meteo_evapotranspiration_grid", 
                        "distance_piezo_meteo", 
                        "hydro_observation_result_elab", 
                        "hydro_longitude",
                        "hydro_latitude", 
                        "distance_piezo_hydro", 
                        "prelev_other_volume_sum", 
                        "insee_%_agri", 
                        "insee_pop_commune", 
                        "insee_%_ind", 
                        "insee_%_const"]

cat_col_features = ["piezo_station_department_code",  
                    "piezo_obtention_mode", 
                    "piezo_status", 
                    "piezo_measure_nature_code", 
                    "hydro_status_label", 
                    "hydro_qualification_code",  
                    "hydro_hydro_quantity_elab"]


# On traite le dataframe features entier
new_features = cp.deepcopy(features)

new_features[num_col_features] = new_features[num_col_features].replace('N/A - division par 0', np.nan)
new_features[num_col_features] = new_features[num_col_features].replace('nan', np.nan)

for col in num_col_features:
    new_features[col].astype("float")

# imputation des valeurs numériques manquantes (np.nan) : 

imputer_mean_posterior = IterativeImputer(random_state=42, sample_posterior=True)

imputer_mean_posterior.fit(new_features[num_col_features])
values_imputed = imputer_mean_posterior.transform(new_features[num_col_features])

new_features[num_col_features] = values_imputed

# # vérif plus de nan en numérique
# nan_count = new_features[num_col_features].isna().sum()
# print(nan_count)
# total_nan_count = new_features[num_col_features].isna().sum().sum()
# print(total_nan_count)

# Gérer les 3 features datetime

new_features['DATE_piezo_measurement_date'] = pd.to_datetime(new_features['piezo_measurement_date'])

new_features['year_piezo_measurement'] = new_features['DATE_piezo_measurement_date'].dt.year
new_features['month_piezo_measurement'] = new_features['DATE_piezo_measurement_date'].dt.month
new_features['day_piezo_measurement'] = new_features['DATE_piezo_measurement_date'].dt.day
new_features = new_features.drop(['piezo_measurement_date'], axis=1)

new_features['DATE_meteo_date'] = pd.to_datetime(new_features['meteo_date'])

new_features['year_meteo_date'] = new_features['DATE_meteo_date'].dt.year
new_features['month_meteo_date'] = new_features['DATE_meteo_date'].dt.month
new_features['day_meteo_date'] = new_features['DATE_meteo_date'].dt.day
new_features = new_features.drop(['meteo_date'], axis=1)


new_features['DATE_hydro_observation_date_elab'] = pd.to_datetime(new_features['hydro_observation_date_elab'])

new_features['year_hydro_observation_date_elab'] = new_features['DATE_hydro_observation_date_elab'].dt.year
new_features['month_hydro_observation_date_elab'] = new_features['DATE_hydro_observation_date_elab'].dt.month
new_features['day_hydro_observation_date_elab'] = new_features['DATE_hydro_observation_date_elab'].dt.day
new_features = new_features.drop(['hydro_observation_date_elab'], axis=1)


new_features_2 = cp.deepcopy(new_features) # au cas où je fais un truc nul


# target_feature = new_features_2[target]

# Ordinal encoding :

ord_cols = ["piezo_obtention_mode", 
            "piezo_status", 
            "piezo_measure_nature_code", 
            "hydro_status_label",
            "hydro_qualification_code",
            "piezo_groundwater_level_category"]

for col in ord_cols:

    temp = new_features_2[col]
    temp = pd.DataFrame(temp)

    ordinal_encoder = OrdinalEncoder()
    ordinal_encoder.fit(temp)
    ordinal_enc = ordinal_encoder.transform(temp)

    new_features_2['ORDINAL_' + col] = ordinal_enc


new_features_2 = new_features_2.drop(ord_cols, axis=1)

# test.sample(50)

# Onehot encoding
onehot_cols = ["piezo_station_department_code", "hydro_hydro_quantity_elab"]
new_features_2 = pd.get_dummies(new_features_2, columns=onehot_cols, drop_first=True)
# new_features_2[target] = target_feature # tout est onehot encodé sauf la target var

# Drop les colonnes avec "DATE" (datetime qui sert plus à rien) :

date_cols = ["DATE_piezo_measurement_date", "DATE_meteo_date", "DATE_hydro_observation_date_elab"]
new_features_2 = new_features_2.drop(date_cols, axis=1)


# On cast toutes les features (maintenant toutes encodées donc numérique ou booléen) 
# en float

for col in new_features_2.columns:
    new_features_2[col] = new_features_2[col].astype("float64")


In [ ]:
pd.set_option('display.max_columns', None)
new_features_2.head()

In [ ]:
new_features_2.info()

### Suite preprocessing

#### Virer les nans et inf qu'il y a encore ?

In [ ]:
# ds = new_features_2.isin([np.inf, -np.inf]) 

# count = np.isinf(new_features_2).values.sum() 
# print("It contains " + str(count) + " infinite values") 

new_features_2.isnull().sum().sum()

On vire les na qui sont apparus (jsp comment mais pour l'instant on va vite ca vire)

In [ ]:
new_features_2 = new_features_2.dropna()

In [ ]:
new_features_2.isnull().sum().sum()


#### Drop row_index sert à rien à part géner

In [ ]:
new_features_2 = new_features_2.drop("row_index", axis=1)

#### Tout doit être en numérique après encoding (pas d'object)

In [ ]:
dfSummary(new_features_2)

#### ordinal encoding

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
ord_cols = ["piezo_obtention_mode", "piezo_status"]
test = new_features_2[ord_cols]
test = pd.DataFrame(test)
test.head()

for col in ord_cols:

    temp = test[col]
    temp = pd.DataFrame(temp)

    ordinal_encoder = OrdinalEncoder()
    ordinal_encoder.fit(temp)
    ordinal_enc = ordinal_encoder.transform(temp)

    test['NUM_' + col] = ordinal_enc

test.sample(50)

ordinal encoding : 

piezo_obtention_mode
1. Valeur mesurée
2. Mode d'obtention inconnu
3. Valeur reconstituée

piezo_status
1. Donnée contrôlée niveau 2
2. Donnée contrôlée niveau 1
3. Donnée brute
4. Donnée interprétée

piezo_measure_nature_code
1. 0
2. N
3. nan
4. I
5. D
6. S

hydro_status_label
1. Donnée validée
2. Donnée pré-validée
3. Donnée brute
4. Donnée corrigée

hydro_qualification_code
1. 20
2. 12
3. 16




one hot : 
piezo_station_department_code
hydro_hydro_quantity_elab


In [ ]:
dfSummary(new_features[cat_col_features])

## Data manipulation

In [ ]:
!pip install umap-learn

In [ ]:
unique_values = new_features_2[ord_target].unique()
print(unique_values)

In [ ]:
# Separate the dependent variable and indepedent variables

ord_target = "ORDINAL_" + target

Y = new_features_2[ord_target]

X = new_features_2.drop(columns=ord_target)


# Robust Scaling de X

from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA

transformer = RobustScaler()
transformer.set_output(transform="pandas")
transformer.fit(X)
X_scaled = transformer.transform(X)
X_scaled

# PCA pour réduire la dimension

pca = PCA()
pca.set_output(transform="pandas")
comp = pca.fit(X_scaled)

plt.plot(np.cumsum(comp.explained_variance_ratio_))
plt.grid()
plt.xlabel('Number of Principal Components')
plt.ylabel('Explained Variance')
sns.despine()

pca = PCA(n_components=15).fit(X_scaled)
X_transformed = pca.transform(X_scaled)

X_transformed = pd.DataFrame(X_transformed)

X_transformed.to_csv("./data/preprocessed_afterPCA_Robust_X_train_10k.csv")


# # Add the intercept term
# X_transformed = sm.add_constant(X_transformed)

# # splitting the data in 70:30 ratio of train to test data
# X_train, X_test, y_train, y_test = train_test_split(X_transformed, Y, test_size = 0.30, random_state=42)

In [ ]:
b = pd.DataFrame(Y)

for col in b.columns:
    b[col] = b[col].astype("int")
b

In [ ]:
# UMAP pour réduire la dimension

import umap.umap_ as umap
reducer = umap.UMAP()
embedding = reducer.fit_transform(X_scaled)
embedding.shape

plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=[sns.color_palette()[x] for x in b[ord_target].map({"0": int(0), 
                                                "1":int(1), 
                                                "2":int(2),
                                                "3":int(3),
                                                "4":int(4),
                                                })])
plt.gca().set_aspect('equal', 'datalim')
plt.title('UMAP projection of the Penguin dataset', fontsize=24)

In [ ]:
X_train

In [ ]:
X_test

## Feature Engineering & Selection

### VIF

In [ ]:
vif = checking_vif(X_train)
vif

On vire les features VIF > 5

In [ ]:
vif[vif["VIF"] < 5]

In [ ]:
features_vif = vif[vif["VIF"] < 5]["feature"].to_numpy()

X_train_2 = X_train[features_vif]
X_test_2 = X_test[features_vif]
X_train_2

In [ ]:
X_test_2

## Predictive Modeling

### XGBoost multiclass classifier

#### Test naïf rapide

In [ ]:
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

import xgboost as xgb

In [ ]:
xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

accuracy_score(y_test, y_pred)

#### GridSearch CV

In [ ]:
from scipy.stats import uniform, randint

def report_best_scores(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [ ]:
params = {
    "colsample_bytree": uniform(0.7, 0.3),
    "gamma": uniform(0, 0.5),
    "learning_rate": uniform(0.03, 0.3), # default 0.1 
    "max_depth": randint(2, 6), # default 3
    "n_estimators": randint(100, 150), # default 100
    "subsample": uniform(0.6, 0.4)
}


search = RandomizedSearchCV(xgb_model, 
                            param_distributions=params, 
                            random_state=42, 
                            n_iter=100, 
                            cv=2, 
                            verbose=1, 
                            n_jobs=1, 
                            return_train_score=True)

search.fit(X_train_2, y_train)




In [ ]:
report_best_scores(search.cv_results_, 1)